In [ ]:
%cd ..
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

# Load an example dataset and example kernels

In [ ]:
images = tfds.load(name="bsds500", split="validation") \
                .map(datasets.get_image) \
                .map(datasets.to_float32) \
                .map(datasets.from_255_to_1_range)
kernels = tfds.load(name='schelten_kernels/dmsp', split='test') \
                .map(datasets.crop_kernel_to_size) \
                .map(datasets.get_value('kernel')) \
                .map(datasets.to_float32) \
                .map(dppp.conv2D_filter_rgb)

In [ ]:
noise_stddev = 0.04
image = datasets.get_one_example(images, index=67)[None,...]
kernel = datasets.get_one_example(kernels, index=0)

# Degrade the image (with wrapping border)

In [ ]:
degraded = dppp.blur(image, kernel, noise_stddev, clip_final=False, mode='wrap')

# Reconstruct the image

In [ ]:
# Load the denoiser
denoiser, (denoiser_min, denoiser_max) = dppp.load_denoiser('models/drugan+_0.0-0.2.h5')

# Callbacks
log_dir = os.path.join('logs', 'ppp_nb')
callbacks = [
    # Print the PSNR every 2nd step
    dppp.callback_print_psnr('psnr', 2, image),
    # Log the SSIM to TensorBoard
    dppp.callback_tb_ssim(log_dir, 'ssim', image)
]

# Reconstruct the image
reconstructed = dppp.hqs_deblur_nb(degraded, kernel, noise_stddev, denoiser, denoiser_max,
                                   callbacks=callbacks)

In [ ]:
# Print PSNR and FSIM
psnr = dppp.psnr(image, reconstructed).numpy()[0]
fsim = dppp.fsim(image, reconstructed).numpy()[0]
print(f"Reconstructed PSNR: {psnr:0.2f}, FSIM: {fsim:0.4f}")

# Visualize
visualize.draw_images([image[0], degraded[0], reconstructed[0]])